In [1]:
import pandas as pd
import glob

In [10]:
dynamic_data_files = glob.glob('../../data/fishing_boats_dynamic/Dynamic_*.csv')
all_boats_trajectories = {}
for dynamic_data_file in dynamic_data_files:
    print(dynamic_data_file)
    df_dynamic = pd.read_csv(dynamic_data_file)
    # print(df_dynamic.head())
    data_grouped = df_dynamic.groupby("MMSI")
    for mmsi, data in data_grouped:
        if mmsi not in all_boats_trajectories:
            all_boats_trajectories[mmsi] = data.copy()  # Create a copy to avoid SettingWithCopyWarning
        else:
            all_boats_trajectories[mmsi] = pd.concat([all_boats_trajectories[mmsi], data], ignore_index=True)



../../data/fishing_boats_dynamic/Dynamic_20230502_fishing_boats.csv
../../data/fishing_boats_dynamic/Dynamic_20230501_fishing_boats.csv


In [11]:
print(len(all_boats_trajectories.keys()))

663


In [45]:
df = all_boats_trajectories[list(all_boats_trajectories.keys())[2]]
df = df.sort_index()

In [46]:
df

,MMSI,Date,Latitude,Longitude,SOG,COG,Heading
0,108672075,2023-05-02 21:23:22,35.247260,122.262020,8.5,330.8,511
1,108672075,2023-05-02 21:24:16,35.248818,122.260813,6.9,327.9,511
2,108672075,2023-05-02 21:24:29,35.249188,122.260505,8.0,329.5,511
3,108672075,2023-05-01 16:55:35,34.765128,122.441865,8.3,170.3,511
4,108672075,2023-05-01 16:55:48,34.764625,122.441920,8.3,173.7,511
5,108672075,2023-05-01 17:00:28,34.758133,122.443578,5.1,359.3,511
6,108672075,2023-05-01 17:52:15,34.837367,122.443537,5.5,358.0,511
7,108672075,2023-05-01 18:10:41,34.866912,122.443363,5.8,359.0,511
8,108672075,2023-05-01 18:32:15,34.900963,122.451562,5.4,26.1,511
9,108672075,2023-05-01 18:33:35,34.902682,122.452918,5.5,32.0,511


In [47]:
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date")
df = df.sort_index()

df_first = df.iloc[:1].copy()
df_first.index = [df.index.min().floor("10T")]

# Duplicate the last row with the time ceiled to the end of the last 10-minute interval
df_last = df.iloc[-1:].copy()
df_last.index = [df.index.max().ceil("10T")]

# Append the modified rows to the original dataframe
df = pd.concat([df_first, df, df_last])

# Define exact 10-minute sampling times
start_time = df.index.min().floor("H")  # Round down to the nearest hour
end_time = df.index.max().ceil("H")  # Round up to the nearest hour
sampling_times = pd.date_range(start_time, end_time, freq="10T")

# Filter only timestamps where at least one real record exists within ±10 minutes
valid_sampling_times = [t for t in sampling_times if any(abs(df.index - t) <= pd.Timedelta(minutes=10))]

# Reindex only valid sampling times
df_interpolated = df.reindex(df.index.union(valid_sampling_times)).sort_index()

# Perform linear interpolation
df_interpolated = df_interpolated.interpolate(method="time")

# Keep only the sampled timestamps and drop any remaining NaNs
df_sampled = df_interpolated.loc[valid_sampling_times].dropna().reset_index()
df_sampled.rename(columns={"Date": "Sampled_Time"}, inplace=True)
df_sampled

/var/folders/xl/2j0v3vnj3gbfd5vvjwwz9lr00000gn/T/ipykernel_86765/413493353.py:6: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_first.index = [df.index.min().floor("10T")]
/var/folders/xl/2j0v3vnj3gbfd5vvjwwz9lr00000gn/T/ipykernel_86765/413493353.py:10: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_last.index = [df.index.max().ceil("10T")]
/var/folders/xl/2j0v3vnj3gbfd5vvjwwz9lr00000gn/T/ipykernel_86765/413493353.py:16: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  start_time = df.index.min().floor("H")  # Round down to the nearest hour
/var/folders/xl/2j0v3vnj3gbfd5vvjwwz9lr00000gn/T/ipykernel_86765/413493353.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  end_time = df.index.max().ceil("H")  # Round up to the nearest hour
/var/folders/xl/2j0v3vnj3gbfd5vvjwwz

,index,MMSI,Latitude,Longitude,SOG,COG,Heading
0,2023-05-01 16:50:00,108672075.0,34.765128,122.441865,8.300000,170.300000,511.0
1,2023-05-01 17:00:00,108672075.0,34.758782,122.443412,5.420000,340.740000,511.0
2,2023-05-01 17:10:00,108672075.0,34.772720,122.443571,5.173640,359.060669,511.0
3,2023-05-01 17:50:00,108672075.0,34.833924,122.443538,5.482620,358.056485,511.0
4,2023-05-01 18:00:00,108672075.0,34.849788,122.443464,5.626130,358.420434,511.0
5,2023-05-01 18:10:00,108672075.0,34.865816,122.443370,5.788879,358.962929,511.0
6,2023-05-01 18:20:00,108672075.0,34.881622,122.446905,5.627202,215.189258,511.0
7,2023-05-01 18:30:00,108672075.0,34.897411,122.450706,5.441731,60.830680,511.0
8,2023-05-01 18:40:00,108672075.0,34.904846,122.455105,4.144119,45.639043,511.0
9,2023-05-01 19:10:00,108672075.0,34.902787,122.457248,1.024020,130.773174,511.0


In [48]:
df.sort_index()

,MMSI,Latitude,Longitude,SOG,COG,Heading
2023-05-01 16:50:00,108672075,34.765128,122.441865,8.3,170.3,511
2023-05-01 16:55:35,108672075,34.765128,122.441865,8.3,170.3,511
2023-05-01 16:55:48,108672075,34.764625,122.441920,8.3,173.7,511
2023-05-01 17:00:28,108672075,34.758133,122.443578,5.1,359.3,511
2023-05-01 17:52:15,108672075,34.837367,122.443537,5.5,358.0,511
2023-05-01 18:10:41,108672075,34.866912,122.443363,5.8,359.0,511
2023-05-01 18:32:15,108672075,34.900963,122.451562,5.4,26.1,511
2023-05-01 18:33:35,108672075,34.902682,122.452918,5.5,32.0,511
2023-05-01 18:35:08,108672075,34.904635,122.454385,4.4,30.0,511
2023-05-01 18:35:37,108672075,34.905147,122.454792,4.6,33.2,511
